In [16]:
from IPython.display import HTML

## Testing recognition from bash - DEMO

In [ ]:
!python mmaction2/demo/demo_skeleton.py ../DJI_0012_720p_15r_10s.MP4 DJI_0012_720p_15r_10s_2019.MP4 \
    --config mmaction2/configs/skeleton/posec3d/slowonly_r50_8xb16-u48-240e_ntu60-xsub-keypoint.py \
    --checkpoint https://download.openmmlab.com/mmaction/skeleton/posec3d/slowonly_r50_u48_240e_ntu60_xsub_keypoint/slowonly_r50_u48_240e_ntu60_xsub_keypoint-f3adabf1.pth \
    --det-config mmaction2/demo/demo_configs/faster-rcnn_r50_fpn_2x_coco_infer.py \
    --det-checkpoint http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth \
    --det-score-thr 0.9 \
    --det-cat-id 0 \
    --pose-config mmaction2/demo/demo_configs/td-hm_hrnet-w32_8xb64-210e_coco-256x192_infer.py \
    --pose-checkpoint https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth \
    --label-map mmaction2/tools/data/skeleton/label_map_ntu60.txt

In [ ]:
HTML('<video width=50% controls autoplay loop><source src="data/uji_jalan.mp4"></video>')

In [ ]:
HTML('<video width=50% controls autoplay loop><source src="data/uji_jalan_out_demo.mp4"></video>')

## Testing recognition from python - DEMO

In [ ]:
import tempfile

import cv2
import mmcv
import mmengine
import torch
from mmengine.utils import track_iter_progress

from mmaction.apis import (detection_inference, inference_skeleton,
                           init_recognizer, pose_inference)
from mmaction.registry import VISUALIZERS
from mmaction.utils import frame_extract

import moviepy.editor as mpy

In [ ]:
import moviepy.config as cf
print ( cf.get_setting("FFMPEG_BINARY") ) # prints the current setting, make sure to use imageio_ffmpeg

In [ ]:
FONTFACE = cv2.FONT_HERSHEY_DUPLEX
FONTSCALE = 0.75
FONTCOLOR = (255, 255, 255)  # BGR, white
THICKNESS = 1
LINETYPE = 1

def visualize(pose_config, out_filename, frames, data_samples, action_label):
    pose_config = mmengine.Config.fromfile(pose_config)
    visualizer = VISUALIZERS.build(pose_config.visualizer)
    visualizer.set_dataset_meta(data_samples[0].dataset_meta)

    vis_frames = []
    print('Drawing skeleton for each frame')
    for d, f in track_iter_progress(list(zip(data_samples, frames))):
        f = mmcv.imconvert(f, 'bgr', 'rgb')
        visualizer.add_datasample(
            'result',
            f,
            data_sample=d,
            draw_gt=False,
            draw_heatmap=True,
            draw_bbox=True,
            show=False,
            wait_time=0,
            out_file=None,
            kpt_thr=0.3)
        vis_frame = visualizer.get_image()
        # heatmap = visualizer.draw_featmap(featmap, img, channel_reduction='select_max')
        cv2.putText(vis_frame, action_label, (10, 30), FONTFACE, FONTSCALE,
                    FONTCOLOR, THICKNESS, LINETYPE)
        vis_frames.append(vis_frame)

    vid = mpy.ImageSequenceClip(vis_frames, fps=15)
    vid.write_videofile(out_filename, remove_temp=True)
    vid.ipython_display()

In [ ]:
video = '../DJI_0014_1080p_15r_10s.MP4'
out_filename = 'DJI_0014_1080p_15r_10s_2019.MP4'

# Choose to use an action classification config
config = 'mmaction2/configs/skeleton/posec3d/slowonly_r50_8xb16-u48-240e_ntu60-xsub-keypoint.py'
# Setup a checkpoint file to load
checkpoint = 'https://download.openmmlab.com/mmaction/skeleton/posec3d/slowonly_r50_u48_240e_ntu60_xsub_keypoint/slowonly_r50_u48_240e_ntu60_xsub_keypoint-f3adabf1.pth'

# human detection config
det_config = 'mmaction2/demo/demo_configs/faster-rcnn_r50_fpn_2x_coco_infer.py'
det_checkpoint = 'http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth'
det_score_thr = 0.9
det_cat_id = 0

# pose estimation config
pose_config = 'mmaction2/demo/demo_configs/td-hm_hrnet-w32_8xb64-210e_coco-256x192_infer.py'
pose_checkpoint = 'https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth'
label_map = 'mmaction2/tools/data/skeleton/label_map_ntu60.txt'

In [ ]:
device = 'cuda:0'
short_side = 480

In [ ]:
tmp_dir = tempfile.TemporaryDirectory()
frame_paths, frames = frame_extract(video, short_side,
                                    tmp_dir.name)

h, w, _ = frames[0].shape

In [ ]:
# Get Human detection results.
det_results, _ = detection_inference(det_config, det_checkpoint,
                                     frame_paths, det_score_thr,
                                     det_cat_id, device)
torch.cuda.empty_cache()

In [ ]:
# Get Pose estimation results.
pose_results, pose_data_samples = pose_inference(pose_config,
                                                 pose_checkpoint,
                                                 frame_paths, det_results,
                                                 device)
torch.cuda.empty_cache()

In [ ]:
# Initialize the recognizer
config = mmengine.Config.fromfile(config)
# build the model from a config file and a checkpoint file
model = init_recognizer(config, checkpoint, device)

# Get Action classification results.
result = inference_skeleton(model, pose_results, (h, w))

In [ ]:
# find the index of highest predicted score on result
max_pred_index = result.pred_score.argmax().item()

label_map = [x.strip() for x in open(label_map).readlines()]

# set the highest predicted label as action_label
action_label = label_map[max_pred_index]
print(action_label)

In [ ]:
visualize(pose_config, out_filename, frames, pose_data_samples, action_label)

tmp_dir.cleanup()

In [ ]:
HTML('<video width=50% controls autoplay loop><source src="data/uji_jalan.mp4"></video>')

In [ ]:
HTML('<video width=50% controls autoplay loop><source src="DJI_0012_720p_15r_2019.MP4"></video>')

## Testing recognition from bash - 2019

In [ ]:
!python mmaction2/demo/demo_skeleton.py data/uji_jalan.mp4 data/uji_jalan_out_2019.mp4 \
    --config mmaction2/configs/skeleton/posec3d/slowonly_r50_u48_240e_ntu120_xsub_keypoint.py \
    --checkpoint work_dirs/slowonly_r50_u48_240e_ntu120_xsub_keypoint/best_top1_acc_epoch_90_8.pth \
    --det-config mmaction2/demo/demo_configs/faster-rcnn_r50_fpn_2x_coco_infer.py \
    --det-checkpoint http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth \
    --det-score-thr 0.9 \
    --det-cat-id 0 \
    --pose-config mmaction2/demo/demo_configs/td-hm_hrnet-w32_8xb64-210e_coco-256x192_infer.py \
    --pose-checkpoint https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth \
    --label-map mmaction2/tools/data/skeleton/label_5.txt

In [ ]:
HTML('<video width=50% controls autoplay loop><source src="data/uji_jalan.mp4"></video>')

In [ ]:
HTML('<video width=50% controls autoplay loop><source src="data/uji_jalan_out_2019.mp4"></video>')

## Testing recognition from python - 2019

In [ ]:
FONTFACE = cv2.FONT_HERSHEY_DUPLEX
FONTSCALE = 0.75
FONTCOLOR = (255, 255, 255)  # BGR, white
THICKNESS = 1
LINETYPE = 1

In [ ]:
def visualize(pose_config, out_filename, frames, data_samples, action_label):
    pose_config = mmengine.Config.fromfile(pose_config)
    visualizer = VISUALIZERS.build(pose_config.visualizer)
    visualizer.set_dataset_meta(data_samples[0].dataset_meta)

    vis_frames = []
    print('Drawing skeleton for each frame')
    for d, f in track_iter_progress(list(zip(data_samples, frames))):
        f = mmcv.imconvert(f, 'bgr', 'rgb')
        visualizer.add_datasample(
            'result',
            f,
            data_sample=d,
            draw_gt=False,
            draw_heatmap=False,
            draw_bbox=True,
            show=False,
            wait_time=0,
            out_file=None,
            kpt_thr=0.3)
        vis_frame = visualizer.get_image()
        cv2.putText(vis_frame, action_label, (10, 30), FONTFACE, FONTSCALE,
                    FONTCOLOR, THICKNESS, LINETYPE)
        vis_frames.append(vis_frame)

    vid = mpy.ImageSequenceClip(vis_frames, fps=24)
    vid.write_videofile(out_filename, remove_temp=True)

In [ ]:
video = 'data/uji_jalan.mp4'
out_filename = 'data/uji_jalan_out_2019.mp4'

# Choose to use an action classification config
config = 'mmaction2/configs/skeleton/posec3d/slowonly_r50_u48_240e_ntu120_xsub_keypoint.py'
# Setup a checkpoint file to load
checkpoint = 'work_dirs/slowonly_r50_u48_240e_ntu120_xsub_keypoint/best_top1_acc_epoch_90_8.pth' #class 5 label

# human detection config
det_config = 'mmaction2/demo/demo_configs/faster-rcnn_r50_fpn_2x_coco_infer.py'
det_checkpoint = 'http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth'
det_score_thr = 0.9
det_cat_id = 0

# pose estimation config
pose_config = 'mmaction2/demo/demo_configs/td-hm_hrnet-w32_8xb64-210e_coco-256x192_infer.py'
pose_checkpoint = 'https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth'
label_map = 'mmaction2/tools/data/skeleton/label_5.txt'

In [ ]:
device = 'cuda:0'
short_side = 480

In [ ]:
tmp_dir = tempfile.TemporaryDirectory()
frame_paths, frames = frame_extract(video, short_side,
                                    tmp_dir.name)

h, w, _ = frames[0].shape

In [ ]:
# Get Human detection results.
det_results, _ = detection_inference(det_config, det_checkpoint,
                                     frame_paths, det_score_thr,
                                     det_cat_id, device)
torch.cuda.empty_cache()

In [ ]:
# Get Pose estimation results.
pose_results, pose_data_samples = pose_inference(pose_config,
                                                 pose_checkpoint,
                                                 frame_paths, det_results,
                                                 device)
torch.cuda.empty_cache()

In [ ]:
# Initialize the recognizer
config = mmengine.Config.fromfile(config)
# build the model from a config file and a checkpoint file
model = init_recognizer(config, checkpoint, device)

# Get Action classification results.
result = inference_skeleton(model, pose_results, (h, w))

In [ ]:
config = 'mmaction2/configs/skeleton/posec3d/slowonly_r50_u48_240e_ntu120_xsub_keypoint.py'
checkpoint = "work_dirs/slowonly_r50_u48_240e_ntu120_xsub_keypoint/best_top1_acc_epoch_90_8.pth"

config = mmengine.Config.fromfile(config)

# build the model from a config file and a checkpoint file
model = init_recognizer(config, checkpoint, device)

# Get Action classification results.
result = inference_skeleton(model, pose_results, (h, w))

In [ ]:
# find the index of highest predicted score on result
max_pred_index = result.pred_score.argmax().item()

label_map = [x.strip() for x in open(label_map).readlines()]

# set the highest predicted label as action_label
action_label = label_map[max_pred_index]
print(action_label)

In [ ]:
visualize(pose_config, out_filename, frames, pose_data_samples, action_label)

tmp_dir.cleanup()

In [ ]:
HTML('<video width=50% controls autoplay loop><source src="data/video_out.mp4"></video>')

## Testing recognition + detection from bash - DEMO

In [ ]:
!python mmaction2/demo/demo_video_structuralize.py \
    --video mmaction2/demo/test_video_structuralize.mp4 \
    --out-filename mmaction2/demo/test_stdet_recognition_output.mp4 \
\
    --rgb-stdet-config mmaction2/configs/detection/slowonly/slowonly_kinetics400-pretrained-r101_8xb16-8x8x1-20e_ava21-rgb.py \
    --skeleton-stdet-checkpoint https://download.openmmlab.com/mmaction/skeleton/posec3d/posec3d_ava.pth \
\
    --det-config mmaction2/demo/demo_configs/faster-rcnn_r50_fpn_2x_coco_infer.py \
    --det-checkpoint http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth \
\
    --pose-config mmaction2/demo/demo_configs/td-hm_hrnet-w32_8xb64-210e_coco-256x192_infer.py \
    --pose-checkpoint https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth \
\
    --skeleton-config mmaction2/configs/skeleton/posec3d/slowonly_r50_8xb16-u48-240e_ntu60-xsub-keypoint.py \
    --skeleton-checkpoint https://download.openmmlab.com/mmaction/skeleton/posec3d/posec3d_k400.pth \
\
    --use-skeleton-stdet \
    --use-skeleton-recog \
\
    --label-map-stdet mmaction2/tools/data/ava/label_map.txt \
    --label-map mmaction2/tools/data/kinetics/label_map_k400.txt

In [ ]:
HTML('<video width=50% controls autoplay loop><source src="mmaction2/demo/test_video_structuralize.mp4"></video>')

In [ ]:
HTML('<video width=50% controls autoplay loop><source src="mmaction2/demo/test_stdet_recognition_output.mp4"></video>')

In [ ]:
!python mmaction2/demo/demo_video_structuralize.py \
    --video mmaction2/demo/demo_skeleton.mp4 \
    --out-filename mmaction2/demo/demo_skeleton_vs_out.mp4 \
    --rgb-stdet-config mmaction2/configs/detection/slowonly/slowonly_kinetics400-pretrained-r101_8xb16-8x8x1-20e_ava21-rgb.py \
    --skeleton-stdet-checkpoint https://download.openmmlab.com/mmaction/skeleton/posec3d/posec3d_ava.pth \
    --det-config mmaction2/demo/demo_configs/faster-rcnn_r50_fpn_2x_coco_infer.py \
    --det-checkpoint http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth \
    --pose-config mmaction2/demo/demo_configs/td-hm_hrnet-w32_8xb64-210e_coco-256x192_infer.py \
    --pose-checkpoint https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth \
    --skeleton-config mmaction2/configs/skeleton/posec3d/slowonly_r50_8xb16-u48-240e_ntu60-xsub-keypoint.py \
    --skeleton-checkpoint https://download.openmmlab.com/mmaction/skeleton/posec3d/posec3d_k400.pth \
    --use-skeleton-stdet \
    --use-skeleton-recog \
    --label-map-stdet mmaction2/tools/data/ava/label_map.txt \
    --label-map mmaction2/tools/data/kinetics/label_map_k400.txt

In [ ]:
HTML('<video width=50% controls autoplay loop><source src="mmaction2/demo/demo_skeleton.mp4"></video>')

In [ ]:
HTML('<video width=50% controls autoplay loop><source src="mmaction2/demo/demo_skeleton_vs_out.mp4"></video>')

In [ ]:
!python mmaction2/demo/demo_video_structuralize.py \
    --video data/DJI_0011.MP4 \
    --out-filename data/DJI_0011_out.MP4 \
\
    --rgb-stdet-config mmaction2/configs/detection/slowonly/slowonly_kinetics400-pretrained-r101_8xb16-8x8x1-20e_ava21-rgb.py \
    --skeleton-stdet-checkpoint https://download.openmmlab.com/mmaction/skeleton/posec3d/posec3d_ava.pth \
\
    --det-config mmaction2/demo/demo_configs/faster-rcnn_r50_fpn_2x_coco_infer.py \
    --det-checkpoint http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth \
\
    --pose-config mmaction2/demo/demo_configs/td-hm_hrnet-w32_8xb64-210e_coco-256x192_infer.py \
    --pose-checkpoint https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth \
\
    --skeleton-config mmaction2/configs/skeleton/posec3d/slowonly_r50_8xb16-u48-240e_ntu60-xsub-keypoint.py \
    --skeleton-checkpoint https://download.openmmlab.com/mmaction/skeleton/posec3d/posec3d_k400.pth \
\
    --use-skeleton-stdet \
    --use-skeleton-recog \
\
    --label-map-stdet mmaction2/tools/data/ava/label_map.txt \
    --label-map mmaction2/tools/data/kinetics/label_map_k400.txt

In [ ]:
HTML('<video width=50% controls autoplay loop><source src="mmaction2/demo/demo_skeleton.mp4"></video>')

In [ ]:
HTML('<video width=50% controls autoplay loop><source src="mmaction2/demo/demo_skeleton_vs_out.mp4"></video>')

## Testing recognition + detection from python - DEMO

In [1]:
# Copyright (c) OpenMMLab. All rights reserved.
import argparse
import copy as cp
import tempfile
import warnings

import cv2
import mmcv
import mmengine
import numpy as np
import torch
from mmengine import DictAction
from mmengine.structures import InstanceData

from mmaction.apis import (detection_inference, inference_recognizer,
                           inference_skeleton, init_recognizer, pose_inference)
from mmaction.registry import VISUALIZERS
from mmaction.structures import ActionDataSample
from mmaction.utils import frame_extract

from mmdet.apis import init_detector
# try:
#     from mmdet.apis import init_detector
# except (ImportError, ModuleNotFoundError):
#     warnings.warn('Failed to import `init_detector` form `mmdet.apis`. '
#                   'These apis are required in skeleton-based applications! ')

import moviepy.editor as mpy
# try:
#     import moviepy.editor as mpy
# except ImportError:
#     raise ImportError('Please install moviepy to enable output file')

In [2]:
FONTFACE = cv2.FONT_HERSHEY_DUPLEX
FONTSCALE = 1
FONTCOLOR = (255, 255, 255)  # BGR, white
MSGCOLOR = (128, 128, 128)  # BGR, gray
THICKNESS = 1
LINETYPE = 1

In [23]:
def hex2color(h):
    """Convert the 6-digit hex string to tuple of 3 int value (RGB)"""
    return (int(h[:2], 16), int(h[2:4], 16), int(h[4:], 16))


PLATEBLUE = '03045e-023e8a-0077b6-0096c7-00b4d8-48cae4'
PLATEBLUE = PLATEBLUE.split('-')
PLATEBLUE = [hex2color(h) for h in PLATEBLUE]
PLATEGREEN = '004b23-006400-007200-008000-38b000-70e000'
PLATEGREEN = PLATEGREEN.split('-')
PLATEGREEN = [hex2color(h) for h in PLATEGREEN]


def visualize(pose_config,
              frames,
              annotations,
              pose_data_samples,
              action_result,
              plate=PLATEBLUE,
              max_num=5):
    """Visualize frames with predicted annotations.

    Args:
        frames (list[np.ndarray]): Frames for visualization, note that
            len(frames) % len(annotations) should be 0.
        annotations (list[list[tuple]]): The predicted spatio-temporal
            detection results.
        pose_data_samples (list[list[PoseDataSample]): The pose results.
        action_result (str): The predicted action recognition results.
        pose_model (nn.Module): The constructed pose model.
        plate (str): The plate used for visualization. Default: PLATEBLUE.
        max_num (int): Max number of labels to visualize for a person box.
            Default: 5.

    Returns:
        list[np.ndarray]: Visualized frames.
    """

    assert max_num + 1 <= len(plate)
    frames_ = cp.deepcopy(frames)
    frames_ = [mmcv.imconvert(f, 'bgr', 'rgb') for f in frames_]
    nf, na = len(frames), len(annotations)
    assert nf % na == 0
    nfpa = len(frames) // len(annotations)
    anno = None
    h, w, _ = frames[0].shape
    scale_ratio = np.array([w, h, w, h])

    # add pose results
    if pose_data_samples:
        pose_config = mmengine.Config.fromfile(pose_config)
        visualizer = VISUALIZERS.build(pose_config.visualizer)
        visualizer.set_dataset_meta(pose_data_samples[0].dataset_meta)
        for i, (d, f) in enumerate(zip(pose_data_samples, frames_)):
            visualizer.add_datasample(
                'result',
                f,
                data_sample=d,
                draw_gt=True,
                draw_pred=True,
                draw_heatmap=True,
                draw_bbox=True,
                show=False,
                wait_time=0,
                out_file=None,
                kpt_thr=0.3)
            frames_[i] = visualizer.get_image()
            cv2.putText(frames_[i], action_result, (10, 30), FONTFACE,
                        FONTSCALE, FONTCOLOR, THICKNESS, LINETYPE)

    for i in range(na):
        anno = annotations[i]
        if anno is None:
            continue
        for j in range(nfpa):
            ind = i * nfpa + j
            frame = frames_[ind]

            # add action result for whole video
            cv2.putText(frame, action_result, (10, 30), FONTFACE, FONTSCALE,
                        FONTCOLOR, THICKNESS, LINETYPE)

            # add spatio-temporal action detection results
            for ann in anno:
                box = ann[0]
                label = ann[1]
                if not len(label):
                    continue
                score = ann[2]
                box = (box * scale_ratio).astype(np.int64)
                st, ed = tuple(box[:2]), tuple(box[2:])
                if not pose_data_samples:
                    cv2.rectangle(frame, st, ed, plate[0], 2)

                for k, lb in enumerate(label):
                    if k >= max_num:
                        break
                    text = abbrev(lb)
                    text = ': '.join([text, f'{score[k]:.3f}'])
                    location = (0 + st[0], 18 + k * 18 + st[1])
                    textsize = cv2.getTextSize(text, FONTFACE, FONTSCALE,
                                               THICKNESS)[0]
                    textwidth = textsize[0]
                    diag0 = (location[0] + textwidth, location[1] - 14)
                    diag1 = (location[0], location[1] + 2)
                    cv2.rectangle(frame, diag0, diag1, plate[k + 1], -1)
                    cv2.putText(frame, text, location, FONTFACE, FONTSCALE,
                                FONTCOLOR, THICKNESS, LINETYPE)

    return frames_

In [4]:
video = '../DJI_0014_1080p_15r_10s.MP4'
out_filename = 'DJI_0014_1080p_15r_10s_out.MP4'

# rgb-based spatio temporal detection config
rgb_stdet_config =  "mmaction2/configs/detection/slowonly/slowonly_kinetics400-pretrained-r101_8xb16-8x8x1-20e_ava21-rgb.py"
rgb_stdet_checkpoint = "https://download.openmmlab.com/mmaction/detection/ava/slowonly_omnisource_pretrained_r101_8x8x1_20e_ava_rgb/slowonly_omnisource_pretrained_r101_8x8x1_20e_ava_rgb_20201217-16378594.pth"

# skeleton-based spatio temporal detection config
skeleton_stdet_checkpoint = "https://download.openmmlab.com/mmaction/skeleton/posec3d/posec3d_ava.pth"

# human detection config
det_config = 'mmaction2/demo/demo_configs/faster-rcnn_r50_fpn_2x_coco_infer.py'
det_checkpoint = 'http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth'
det_score_thr = 0.9
#det_cat_id = 0

# pose estimation config
pose_config = 'mmaction2/demo/demo_configs/td-hm_hrnet-w32_8xb64-210e_coco-256x192_infer.py'
pose_checkpoint = 'https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth'

# action classification config
skeleton_config = "mmaction2/configs/skeleton/posec3d/slowonly_r50_8xb16-u48-240e_ntu60-xsub-keypoint.py"
skeleton_checkpoint = "https://download.openmmlab.com/mmaction/skeleton/posec3d/posec3d_k400.pth"
action_score_thr = 0.4

# use skeleton-based method
use_skeleton_stdet = True
use_skeleton_recog = True

label_map_stdet = "mmaction2/tools/data/ava/label_map.txt"
label_map = "mmaction2/tools/data/kinetics/label_map_k400.txt"


# rgb-based action recognition config
rgb_config = "configs/recognition/tsn/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb.py"
rgb_checkpoint = "https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth"

predict_stepsize = 5  # give out a spatio-temporal detection prediction per n frames
output_stepsize = 1  # show one frame per n frames in the demo, we should have: predict_stepsize % output_stepsize == 0
output_fps = 15  # the fps of demo video output

In [5]:
device = 'cuda:0'

In [6]:
# def parse_args():
#     parser = argparse.ArgumentParser(description='MMAction2 demo')
#     parser.add_argument(
#         '--cfg-options',
#         nargs='+',
#         action=DictAction,
#         default={},
#         help='override some settings in the used config, the key-value pair '
#         'in xxx=yyy format will be merged into config file. For example, '
#         "'--cfg-options model.backbone.depth=18 model.backbone.with_cp=True'")
#     args = parser.parse_args()
#     return args

In [7]:
def load_label_map(file_path):
    """Load Label Map.

    Args:
        file_path (str): The file path of label map.

    Returns:
        dict: The label map (int -> label name).
    """
    lines = open(file_path).readlines()
    lines = [x.strip().split(': ') for x in lines]
    return {int(x[0]): x[1] for x in lines}


def abbrev(name):
    """Get the abbreviation of label name:

    'take (an object) from (a person)' -> 'take ... from ...'
    """
    while name.find('(') != -1:
        st, ed = name.find('('), name.find(')')
        name = name[:st] + '...' + name[ed + 1:]
    return name


def pack_result(human_detection, result, img_h, img_w):
    """Short summary.

    Args:
        human_detection (np.ndarray): Human detection result.
        result (type): The predicted label of each human proposal.
        img_h (int): The image height.
        img_w (int): The image width.

    Returns:
        tuple: Tuple of human proposal, label name and label score.
    """
    human_detection[:, 0::2] /= img_w
    human_detection[:, 1::2] /= img_h
    results = []
    if result is None:
        return None
    for prop, res in zip(human_detection, result):
        res.sort(key=lambda x: -x[1])
        results.append(
            (prop.data.cpu().numpy(), [x[0] for x in res], [x[1]
                                                            for x in res]))
    return results


def expand_bbox(bbox, h, w, ratio=1.25):
    x1, y1, x2, y2 = bbox
    center_x = (x1 + x2) // 2
    center_y = (y1 + y2) // 2
    width = x2 - x1
    height = y2 - y1

    square_l = max(width, height)
    new_width = new_height = square_l * ratio

    new_x1 = max(0, int(center_x - new_width / 2))
    new_x2 = min(int(center_x + new_width / 2), w)
    new_y1 = max(0, int(center_y - new_height / 2))
    new_y2 = min(int(center_y + new_height / 2), h)
    return (new_x1, new_y1, new_x2, new_y2)


def cal_iou(box1, box2):
    xmin1, ymin1, xmax1, ymax1 = box1
    xmin2, ymin2, xmax2, ymax2 = box2

    s1 = (xmax1 - xmin1) * (ymax1 - ymin1)
    s2 = (xmax2 - xmin2) * (ymax2 - ymin2)

    xmin = max(xmin1, xmin2)
    ymin = max(ymin1, ymin2)
    xmax = min(xmax1, xmax2)
    ymax = min(ymax1, ymax2)

    w = max(0, xmax - xmin)
    h = max(0, ymax - ymin)
    intersect = w * h
    union = s1 + s2 - intersect
    iou = intersect / union

    return iou


def skeleton_based_action_recognition(skeleton_config, skeleton_checkpoint, device, label_map, pose_results, h, w):
    label_map = [x.strip() for x in open(label_map).readlines()]
    num_class = len(label_map)

    skeleton_config = mmengine.Config.fromfile(skeleton_config)
    skeleton_config.model.cls_head.num_classes = num_class  # for K400 dataset

    skeleton_model = init_recognizer(
        skeleton_config, skeleton_checkpoint, device=device)
    result = inference_skeleton(skeleton_model, pose_results, (h, w))
    action_idx = result.pred_score.argmax().item()
    return label_map[action_idx]


def rgb_based_action_recognition(rgb_config, rgb_checkpoint, device, video, label_map):
    rgb_config = mmengine.Config.fromfile(rgb_config)
    rgb_config.model.backbone.pretrained = None
    rgb_model = init_recognizer(rgb_config, rgb_checkpoint, device)
    action_results = inference_recognizer(rgb_model, video)
    rgb_action_result = action_results.pred_score.argmax().item()
    label_map = [x.strip() for x in open(label_map).readlines()]
    return label_map[rgb_action_result]

def skeleton_based_stdet(predict_stepsize, skeleton_config, skeleton_stdet_checkpoint, device, action_score_thr, label_map, human_detections, pose_results,
                         num_frame, clip_len, frame_interval, h, w):
    window_size = clip_len * frame_interval
    assert clip_len % 2 == 0, 'We would like to have an even clip_len'
    timestamps = np.arange(window_size // 2, num_frame + 1 - window_size // 2,
                           predict_stepsize)

    skeleton_config = mmengine.Config.fromfile(skeleton_config)
    num_class = max(label_map.keys()) + 1  # for AVA dataset (81)
    skeleton_config.model.cls_head.num_classes = num_class
    skeleton_stdet_model = init_recognizer(skeleton_config,
                                           skeleton_stdet_checkpoint,
                                           device)

    skeleton_predictions = []

    print('Performing SpatioTemporal Action Detection for each clip')
    prog_bar = mmengine.ProgressBar(len(timestamps))
    for timestamp in timestamps:
        proposal = human_detections[timestamp - 1]
        if proposal.shape[0] == 0:  # no people detected
            skeleton_predictions.append(None)
            continue

        start_frame = timestamp - (clip_len // 2 - 1) * frame_interval
        frame_inds = start_frame + np.arange(0, window_size, frame_interval)
        frame_inds = list(frame_inds - 1)
        num_frame = len(frame_inds)  # 30

        pose_result = [pose_results[ind] for ind in frame_inds]

        skeleton_prediction = []
        for i in range(proposal.shape[0]):  # num_person
            skeleton_prediction.append([])

            fake_anno = dict(
                frame_dict='',
                label=-1,
                img_shape=(h, w),
                origin_shape=(h, w),
                start_index=0,
                modality='Pose',
                total_frames=num_frame)
            num_person = 1

            num_keypoint = 17
            keypoint = np.zeros(
                (num_person, num_frame, num_keypoint, 2))  # M T V 2
            keypoint_score = np.zeros(
                (num_person, num_frame, num_keypoint))  # M T V

            # pose matching
            person_bbox = proposal[i][:4]
            area = expand_bbox(person_bbox, h, w)

            for j, poses in enumerate(pose_result):  # num_frame
                max_iou = float('-inf')
                index = -1
                if len(poses['keypoints']) == 0:
                    continue
                for k, bbox in enumerate(poses['bboxes']):
                    iou = cal_iou(bbox, area)
                    if max_iou < iou:
                        index = k
                        max_iou = iou
                keypoint[0, j] = poses['keypoints'][index]
                keypoint_score[0, j] = poses['keypoint_scores'][index]

            fake_anno['keypoint'] = keypoint
            fake_anno['keypoint_score'] = keypoint_score

            output = inference_recognizer(skeleton_stdet_model, fake_anno)
            # for multi-label recognition
            score = output.pred_score.tolist()
            for k in range(len(score)):  # 81
                if k not in label_map:
                    continue
                if score[k] > action_score_thr:
                    skeleton_prediction[i].append((label_map[k], score[k]))

        skeleton_predictions.append(skeleton_prediction)
        prog_bar.update()

    return timestamps, skeleton_predictions


def rgb_based_stdet(rgb_stdet_config, rgb_stdet_checkpoint, device, action_score_thr, predict_stepsize, frames, label_map, human_detections, w, h, new_w,
                    new_h, w_ratio, h_ratio):

    rgb_stdet_config = mmengine.Config.fromfile(rgb_stdet_config)
    # rgb_stdet_config.merge_from_dict(args.cfg_options)

    val_pipeline = rgb_stdet_config.val_pipeline
    sampler = [x for x in val_pipeline if x['type'] == 'SampleAVAFrames'][0]
    clip_len, frame_interval = sampler['clip_len'], sampler['frame_interval']
    assert clip_len % 2 == 0, 'We would like to have an even clip_len'

    window_size = clip_len * frame_interval
    num_frame = len(frames)
    # Note that it's 1 based here
    timestamps = np.arange(window_size // 2, num_frame + 1 - window_size // 2,
                           predict_stepsize)

    # Get img_norm_cfg
    img_norm_cfg = dict(
        mean=np.array(rgb_stdet_config.model.data_preprocessor.mean),
        std=np.array(rgb_stdet_config.model.data_preprocessor.std),
        to_rgb=False)

    # Build STDET model
    try:
        # In our spatiotemporal detection demo, different actions should have
        # the same number of bboxes.
        rgb_stdet_config['model']['test_cfg']['rcnn'] = dict(action_thr=0)
    except KeyError:
        pass

    rgb_stdet_config.model.backbone.pretrained = None
    rgb_stdet_model = init_detector(
        rgb_stdet_config, rgb_stdet_checkpoint, device=device)

    predictions = []

    print('Performing SpatioTemporal Action Detection for each clip')
    prog_bar = mmengine.ProgressBar(len(timestamps))
    # for timestamp, proposal in zip(timestamps, human_detections):
    for timestamp in timestamps:
        proposal = human_detections[timestamp - 1]
        if proposal.shape[0] == 0:
            predictions.append(None)
            continue

        start_frame = timestamp - (clip_len // 2 - 1) * frame_interval
        frame_inds = start_frame + np.arange(0, window_size, frame_interval)
        frame_inds = list(frame_inds - 1)

        imgs = [frames[ind].astype(np.float32) for ind in frame_inds]
        _ = [mmcv.imnormalize_(img, **img_norm_cfg) for img in imgs]
        # THWC -> CTHW -> 1CTHW
        input_array = np.stack(imgs).transpose((3, 0, 1, 2))[np.newaxis]
        input_tensor = torch.from_numpy(input_array).to(device)

        datasample = ActionDataSample()
        datasample.proposals = InstanceData(bboxes=proposal)
        datasample.set_metainfo(dict(img_shape=(new_h, new_w)))
        with torch.no_grad():
            result = rgb_stdet_model(
                input_tensor, [datasample], mode='predict')
            scores = result[0].pred_instances.scores
            prediction = []
            # N proposals
            for i in range(proposal.shape[0]):
                prediction.append([])
            # Perform action score thr
            for i in range(scores.shape[1]):
                if i not in label_map:
                    continue
                for j in range(proposal.shape[0]):
                    if scores[j, i] > action_score_thr:
                        prediction[j].append((label_map[i], scores[j,
                                                                   i].item()))
            predictions.append(prediction)
        prog_bar.update()

    return timestamps, predictions

In [8]:
#args = parse_args()
tmp_dir = tempfile.TemporaryDirectory()
frame_paths, original_frames = frame_extract(
    video, out_dir=tmp_dir.name)
num_frame = len(frame_paths)
h, w, _ = original_frames[0].shape

In [9]:
# Get Human detection results and pose results
human_detections, _ = detection_inference(
    det_config,
    det_checkpoint,
    frame_paths,
    det_score_thr,
    device=device)
torch.cuda.empty_cache()
pose_datasample = None
if use_skeleton_recog or use_skeleton_stdet:
    pose_results, pose_datasample = pose_inference(
        pose_config,
        pose_checkpoint,
        frame_paths,
        human_detections,
        device=device)
torch.cuda.empty_cache()

Loads checkpoint by http backend from path: http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth
Performing Human Detection for each frame
[                                                  ] 0/152, elapsed: 0s, ETA:03/01 12:46:07 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
03/01 12:46:07 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 152/152, 18.6 task/s, elapsed: 8s, ETA:     0s
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth
Performing Human Pose Estimation for each frame
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 152/152, 13.5 task/s, elapsed: 11s, ETA:     0s


In [10]:
# resize frames to shortside 256
# new_w, new_h = mmcv.rescale_size((w, h), (480, np.Inf))
new_w, new_h = w, h
# frames = [mmcv.imresize(img, (new_w, new_h)) for img in original_frames]
frames = original_frames
w_ratio, h_ratio = new_w / w, new_h / h

In [11]:
# Load spatio-temporal detection label_map
stdet_label_map = load_label_map(label_map_stdet)
rgb_stdet_config = mmengine.Config.fromfile(rgb_stdet_config)
#rgb_stdet_config.merge_from_dict(args.cfg_options)
try:
    if rgb_stdet_config['data']['train']['custom_classes'] is not None:
        stdet_label_map = {
            id + 1: stdet_label_map[cls]
            for id, cls in enumerate(rgb_stdet_config['data']['train']
                                     ['custom_classes'])
        }
except KeyError:
    pass

In [12]:
action_result = None
if use_skeleton_recog:
    print('Use skeleton-based recognition')
    action_result = skeleton_based_action_recognition(
        skeleton_config, skeleton_checkpoint, device, label_map, pose_results, h, w)
else:
    print('Use rgb-based recognition')
    action_result = rgb_based_action_recognition(rgb_config, rgb_checkpoint, device, video, label_map)

Use skeleton-based recognition
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmaction/skeleton/posec3d/posec3d_k400.pth


In [13]:
stdet_preds = None
if use_skeleton_stdet:
    print('Use skeleton-based SpatioTemporal Action Detection')
    clip_len, frame_interval = 30, 1
    timestamps, stdet_preds = skeleton_based_stdet(predict_stepsize,
                                                   skeleton_config,
                                                   skeleton_stdet_checkpoint,
                                                   device,
                                                   action_score_thr,
                                                   stdet_label_map,
                                                   human_detections,
                                                   pose_results, num_frame,
                                                   clip_len,
                                                   frame_interval, h, w)
    for i in range(len(human_detections)):
        det = human_detections[i]
        det[:, 0:4:2] *= w_ratio
        det[:, 1:4:2] *= h_ratio
        human_detections[i] = torch.from_numpy(det[:, :4]).to(device)

else:
    print('Use rgb-based SpatioTemporal Action Detection')
    for i in range(len(human_detections)):
        det = human_detections[i]
        det[:, 0:4:2] *= w_ratio
        det[:, 1:4:2] *= h_ratio
        human_detections[i] = torch.from_numpy(det[:, :4]).to(device)
    timestamps, stdet_preds = rgb_based_stdet(rgb_stdet_config,
                                              rgb_stdet_checkpoint,
                                              device,
                                              action_score_thr,
                                              predict_stepsize,
                                              frames,
                                              stdet_label_map,
                                              human_detections, w, h,
                                              new_w, new_h, w_ratio,
                                              h_ratio)

Use skeleton-based SpatioTemporal Action Detection
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmaction/skeleton/posec3d/posec3d_ava.pth
Performing SpatioTemporal Action Detection for each clip
[>>>>>>>>>>>>>>>>>>>>>>>>        ] 19/25, 1.9 task/s, elapsed: 10s, ETA:     3s

In [14]:
stdet_results = []
for timestamp, prediction in zip(timestamps, stdet_preds):
    human_detection = human_detections[timestamp - 1]
    stdet_results.append(
        pack_result(human_detection, prediction, new_h, new_w))

def dense_timestamps(timestamps, n):
    """Make it nx frames."""
    old_frame_interval = (timestamps[1] - timestamps[0])
    start = timestamps[0] - old_frame_interval / n * (n - 1) / 2
    new_frame_inds = np.arange(
        len(timestamps) * n) * old_frame_interval / n + start
    return new_frame_inds.astype(np.int64)

dense_n = int(predict_stepsize / output_stepsize)
output_timestamps = dense_timestamps(timestamps, dense_n)
frames = [
    cv2.imread(frame_paths[timestamp - 1])
    for timestamp in output_timestamps
]

if use_skeleton_recog or use_skeleton_stdet:
    pose_datasample = [
        pose_datasample[timestamp - 1] for timestamp in output_timestamps
    ]

In [24]:
vis_frames = visualize(pose_config, frames, stdet_results, pose_datasample,
                       action_result)
vid = mpy.ImageSequenceClip(vis_frames, fps=output_fps)
vid.write_videofile(out_filename)
#tmp_dir.cleanup()

Moviepy - Building video DJI_0014_1080p_15r_10s_out.MP4.
Moviepy - Writing video DJI_0014_1080p_15r_10s_out.MP4



Moviepy - Done !
Moviepy - video ready DJI_0014_1080p_15r_10s_out.MP4


In [ ]:
vis_frames = visualize(pose_config, original_frames, [None], pose_datasample,
                       None)
vid = mpy.ImageSequenceClip(vis_frames, fps=output_fps)
vid.write_videofile(out_filename)
#tmp_dir.cleanup()

In [ ]:
HTML('<video width=50% controls autoplay loop><source src="data/DJI_0011_720p_30r_10s_1.MP4"></video>')

In [25]:
HTML('<video width=50% controls autoplay loop><source src="DJI_0014_1080p_15r_10s_out.MP4"></video>')

In [ ]:
stdet_results

In [ ]:
pose_datasample

In [ ]:
stdet_results = [[([0.34738404, 0.22310092, 0.48000246, 0.7725167 ],
   ['berjalan'],
   [0.5826629996299744]),
  ([0.01663231, 0.46333078, 0.21318212, 0.9706198 ],
   ['berjalan'],
   [0.5030158162117004])],

 [([0.32945454, 0.20106745, 0.4632955 , 0.7321375 ],
   ['berjalan'],
   [0.7553293108940125]),
  ([0.03508885, 0.5091949 , 0.22391675, 0.8820821 ],
   ['berjalan'],
   [0.4826629996299744])],

 [([0.32349333, 0.17711236, 0.4321191 , 0.65161675],
   ['berjalan'],
   [0.9032831192016602]),
  ([0.10454163, 0.5220998 , 0.3504752 , 0.97712123],
   ['membidik'],
   [0.7553293108940125])],

 [([0.2644745 , 0.16871208, 0.39515644, 0.6076856 ],
   ['berjalan'],
   [0.6126652956008911]),
  ([0.19803259, 0.5492611 , 0.33335972, 0.9991328 ],
   ['membidik'],
   [0.6826629996299744])],

 [([0.24765794, 0.1580198 , 0.37136173, 0.65539306],
   ['berjalan'],
   [0.7585893869400024]),
  ([0.28665444, 0.528761  , 0.4399466 , 0.976069  ],
   ['membidik', 'berjalan'],
   [0.5424362421035767, 0.5826629996299744])],

 [([0.22138917, 0.16182165, 0.3362417 , 0.6031747 ],
   ['berjalan'],
   [0.42768335342407227]),
  ([0.34607008, 0.47720572, 0.49217388, 0.96492857],
   ['membidik', 'berjalan'],
   [0.6033088564872742, 0.4358972566299744])],

 [([0.18971214, 0.16396755, 0.30814248, 0.65183   ],
   ['membidik'],
   [0.5126482844352722]),
  ([0.3960483 , 0.41720578, 0.53863937, 0.9650495 ],
   ['membidik', 'berjalan'],
   [0.4544826149940491, 0.5304656096299744])],

 [([0.18906213, 0.15210617, 0.30729657, 0.608378  ],
   ['membidik'],
   [0.8877660632133484]),
  ([0.44944507, 0.38048965, 0.58530056, 0.9249272 ],
   ['membidik', 'berjalan'],
   [0.4524362421035767, 0.6349457854522334])],

 [([0.18576823, 0.16220339, 0.30933827, 0.6383328 ],
   ['membidik'],
   [0.970620334148407]),
  ([0.45537382, 0.3679413 , 0.58573556, 0.9485067 ],
   ['membidik'],
   [0.6524362421035767])],

 [([0.17717798, 0.15202911, 0.3097127 , 0.65155053],
   ['membidik'],
   [0.9800529479980469]),
  ([0.44752756, 0.308973  , 0.569137  , 0.89479536],
   [],
   [])],

 [([0.18266118, 0.16554728, 0.30675864, 0.62944895],
   ['membidik'],
   [0.9602543115615845]),
  ([0.44340008, 0.30527505, 0.5780876 , 0.88293546],
   [],
   [])],

 [([0.19051774, 0.15272126, 0.3065299 , 0.60876125],
   ['membidik'],
   [0.7655255198478699]),
  ([0.42363307, 0.31554592, 0.56499785, 0.9318138 ],
   ['berjalan'],
   [0.42768335342407227])],

 [([0.20385337, 0.16238193, 0.313155  , 0.6106909 ],
   [],
   []),
  ([0.4109021 , 0.3365323 , 0.5192115 , 0.94164234],
   ['berjalan'],
   [0.7224847674369812])],

 [([0.17934328, 0.1823791 , 0.31677333, 0.6556542 ],
   ['berjalan'],
   [0.828754723072052]),
  ([0.35352027, 0.31885496, 0.48454002, 0.9214882 ],
   ['membidik'],
   [0.5471435189247131])],

 [([0.3389372 , 0.31119257, 0.46443167, 0.9064811 ],
   ['membidik', 'berjalan'],
   [0.8655255198478699, 0.8604634404182434]),
  ([0.13453774, 0.1848279 , 0.22745821, 0.65507144],
   ['berjalan'],
   [0.4819600284099579])],

 [([0.07503366, 0.21986169, 0.21571878, 0.75649256],
   ['berjalan'],
   [0.6483674049377441]),
  ([0.27768788, 0.30960205, 0.43776336, 0.93241835],
   ['membidik'],
   [0.7217583775520325])],

 [([0.24384162, 0.309836  , 0.38793728, 0.9277533 ],
   ['membidik'],
   [0.7471435189247131]),
  ([0.00849363, 0.25388023, 0.14361295, 0.7934024 ],
   ['berjalan'],
   [0.5879366397857666])],

 [([0.26038128, 0.32210907, 0.39623025, 0.8983851 ],
   ['membidik'],
   [0.6819600284099579]),
  ([0.        , 0.38365027, 0.06890473, 0.885707  ],
   ['berjalan'],
   [0.4471435189247131])],

 [([0.2863469 , 0.30740547, 0.402365  , 0.8521168 ],
   ['membidik'],
   [0.4826629996299744])],

 [([0.29458097, 0.29107833, 0.43426695, 0.87181914],
   ['membidik'],
   [0.42408162355422974])],

 [([0.29197636, 0.25811318, 0.39831966, 0.73899937],
   ['membidik'],
   [0.41635599732398987]),
  ([0.02044248, 0.38083506, 0.0526862 , 0.48335442],
   [],
   [])],

 [([0.28950912, 0.23450229, 0.3972092 , 0.77767175],
   ['membidik'],
   [0.41037240624427795])],

 [([0.27163148, 0.21188514, 0.38974494, 0.71808577],
   ['membidik', 'berjalan'],
   [0.4826835095882416, 0.565538647679758789])],

 [([0.2699414 , 0.18042147, 0.37456167, 0.7033833 ],
   [],
   []),
  ([0.2733366 , 0.18784562, 0.36866525, 0.47048444],
   ['membidik', 'berjalan'],
   [0.4917432963848114, 0.645386757878687879])],

 [([0.27005124, 0.17286462, 0.38190398, 0.6655994 ],
   ['membidik', 'berjalan'],
   [0.45405617336300354, 0.42127156257629395])],

 [([0.24764012, 0.16216156, 0.36022428, 0.6380821 ],
   ['membidik', 'berjalan'],
   [0.4616035521030426, 0.658987657657989879])],

 [([0.21482459, 0.16565847, 0.33908463, 0.65550977],
   ['membidik', 'berjalan'],
   [0.46344656845668446, 0.44584647698686968])],

 [([0.18005927, 0.15882166, 0.29648805, 0.65959126],
   ['membidik', 'berjalan'],
   [0.6442964545649344, 0.56456648486576889]),
  ([0.27157024, 0.6377393 , 0.39111364, 0.9959626 ],
   ['berjalan'],
   [0.4221991002559662])]]

In [ ]:
pose_datasample

In [ ]:
action_result = "baku tembak"

In [ ]:
frames

In [ ]:
frame_extract(
    "data/DJI_0011_720p_30r_10s_1_out.MP4", out_dir="../../skripsi/extracted/")

In [ ]:
video = 'data/video_2.mp4'
out_filename = 'data/video_2_out.MP4'

# rgb-based spatio temporal detection config
rgb_stdet_config =  "mmaction2/configs/detection/slowonly/slowonly_kinetics400-pretrained-r101_8xb16-8x8x1-20e_ava21-rgb.py"
rgb_stdet_checkpoint = "https://download.openmmlab.com/mmaction/detection/ava/slowonly_omnisource_pretrained_r101_8x8x1_20e_ava_rgb/slowonly_omnisource_pretrained_r101_8x8x1_20e_ava_rgb_20201217-16378594.pth"

# skeleton-based spatio temporal detection config
skeleton_stdet_checkpoint = "https://download.openmmlab.com/mmaction/skeleton/posec3d/posec3d_ava.pth"

# human detection config
det_config = 'mmaction2/demo/demo_configs/faster-rcnn_r50_fpn_2x_coco_infer.py'
det_checkpoint = 'http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth'
det_score_thr = 0.9
#det_cat_id = 0

# pose estimation config
pose_config = 'mmaction2/demo/demo_configs/td-hm_hrnet-w32_8xb64-210e_coco-256x192_infer.py'
pose_checkpoint = 'https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth'

# action classification config
skeleton_config = "mmaction2/configs/skeleton/posec3d/slowonly_r50_8xb16-u48-240e_ntu60-xsub-keypoint.py"
skeleton_checkpoint = "https://download.openmmlab.com/mmaction/skeleton/posec3d/posec3d_k400.pth"
action_score_thr = 0.4

# use skeleton-based method
use_skeleton_stdet = True
use_skeleton_recog = True

label_map_stdet = "mmaction2/tools/data/ava/label_map.txt"
label_map = "mmaction2/tools/data/kinetics/label_map_k400.txt"


# rgb-based action recognition config
rgb_config = "configs/recognition/tsn/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb.py"
rgb_checkpoint = "https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth"

predict_stepsize = 10  # give out a spatio-temporal detection prediction per n frames
output_stepsize = 1  # show one frame per n frames in the demo, we should have: predict_stepsize % output_stepsize == 0
output_fps = 15  # the fps of demo video output






#args = parse_args()
tmp_dir = tempfile.TemporaryDirectory()
frame_paths, original_frames = frame_extract(
    video, out_dir=tmp_dir.name)
num_frame = len(frame_paths)
h, w, _ = original_frames[0].shape







# Get Human detection results and pose results
human_detections, _ = detection_inference(
    det_config,
    det_checkpoint,
    frame_paths,
    det_score_thr,
    device=device)
torch.cuda.empty_cache()
pose_datasample = None
if use_skeleton_recog or use_skeleton_stdet:
    pose_results, pose_datasample = pose_inference(
        pose_config,
        pose_checkpoint,
        frame_paths,
        human_detections,
        device=device)
torch.cuda.empty_cache()

In [ ]:
print(human_detections)

In [ ]:
print(pose_datasample)

In [ ]:
# resize frames to shortside 256
new_w, new_h = mmcv.rescale_size((w, h), (256, np.Inf))
frames = [mmcv.imresize(img, (new_w, new_h)) for img in original_frames]
w_ratio, h_ratio = new_w / w, new_h / h



# Load spatio-temporal detection label_map
stdet_label_map = load_label_map(label_map_stdet)
rgb_stdet_config = mmengine.Config.fromfile(rgb_stdet_config)
#rgb_stdet_config.merge_from_dict(args.cfg_options)
try:
    if rgb_stdet_config['data']['train']['custom_classes'] is not None:
        stdet_label_map = {
            id + 1: stdet_label_map[cls]
            for id, cls in enumerate(rgb_stdet_config['data']['train']
                                     ['custom_classes'])
        }
except KeyError:
    pass






action_result = None
if use_skeleton_recog:
    print('Use skeleton-based recognition')
    action_result = skeleton_based_action_recognition(
        skeleton_config, skeleton_checkpoint, device, label_map, pose_results, h, w)
else:
    print('Use rgb-based recognition')
    action_result = rgb_based_action_recognition(rgb_config, rgb_checkpoint, device, video, label_map)





stdet_preds = None
if use_skeleton_stdet:
    print('Use skeleton-based SpatioTemporal Action Detection')
    clip_len, frame_interval = 30, 1
    timestamps, stdet_preds = skeleton_based_stdet(predict_stepsize,
                                                   skeleton_config,
                                                   skeleton_stdet_checkpoint,
                                                   device,
                                                   action_score_thr,
                                                   stdet_label_map,
                                                   human_detections,
                                                   pose_results, num_frame,
                                                   clip_len,
                                                   frame_interval, h, w)
    for i in range(len(human_detections)):
        det = human_detections[i]
        det[:, 0:4:2] *= w_ratio
        det[:, 1:4:2] *= h_ratio
        human_detections[i] = torch.from_numpy(det[:, :4]).to(device)

else:
    print('Use rgb-based SpatioTemporal Action Detection')
    for i in range(len(human_detections)):
        det = human_detections[i]
        det[:, 0:4:2] *= w_ratio
        det[:, 1:4:2] *= h_ratio
        human_detections[i] = torch.from_numpy(det[:, :4]).to(device)
    timestamps, stdet_preds = rgb_based_stdet(rgb_stdet_config,
                                              rgb_stdet_checkpoint,
                                              device,
                                              action_score_thr,
                                              predict_stepsize,
                                              frames,
                                              stdet_label_map,
                                              human_detections, w, h,
                                              new_w, new_h, w_ratio,
                                              h_ratio)






stdet_results = []
for timestamp, prediction in zip(timestamps, stdet_preds):
    human_detection = human_detections[timestamp - 1]
    stdet_results.append(
        pack_result(human_detection, prediction, new_h, new_w))

def dense_timestamps(timestamps, n):
    """Make it nx frames."""
    old_frame_interval = (timestamps[1] - timestamps[0])
    start = timestamps[0] - old_frame_interval / n * (n - 1) / 2
    new_frame_inds = np.arange(
        len(timestamps) * n) * old_frame_interval / n + start
    return new_frame_inds.astype(np.int64)

dense_n = int(predict_stepsize / output_stepsize)
output_timestamps = dense_timestamps(timestamps, dense_n)
frames = [
    cv2.imread(frame_paths[timestamp - 1])
    for timestamp in output_timestamps
]

if use_skeleton_recog or use_skeleton_stdet:
    pose_datasample = [
        pose_datasample[timestamp - 1] for timestamp in output_timestamps
    ]

In [ ]:
print(stdet_results)

In [ ]:
vis_frames = visualize(pose_config, frames, stdet_results, pose_datasample,
                       action_result)
vid = mpy.ImageSequenceClip(vis_frames, fps=output_fps)
vid.write_videofile(out_filename)
#tmp_dir.cleanup()

In [ ]:
video = 'data/video_3.mp4'
out_filename = 'data/video_3_out.MP4'

# rgb-based spatio temporal detection config
rgb_stdet_config =  "mmaction2/configs/detection/slowonly/slowonly_kinetics400-pretrained-r101_8xb16-8x8x1-20e_ava21-rgb.py"
rgb_stdet_checkpoint = "https://download.openmmlab.com/mmaction/detection/ava/slowonly_omnisource_pretrained_r101_8x8x1_20e_ava_rgb/slowonly_omnisource_pretrained_r101_8x8x1_20e_ava_rgb_20201217-16378594.pth"

# skeleton-based spatio temporal detection config
skeleton_stdet_checkpoint = "https://download.openmmlab.com/mmaction/skeleton/posec3d/posec3d_ava.pth"

# human detection config
det_config = 'mmaction2/demo/demo_configs/faster-rcnn_r50_fpn_2x_coco_infer.py'
det_checkpoint = 'http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth'
det_score_thr = 0.9
#det_cat_id = 0

# pose estimation config
pose_config = 'mmaction2/demo/demo_configs/td-hm_hrnet-w32_8xb64-210e_coco-256x192_infer.py'
pose_checkpoint = 'https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth'

# action classification config
skeleton_config = "mmaction2/configs/skeleton/posec3d/slowonly_r50_8xb16-u48-240e_ntu60-xsub-keypoint.py"
skeleton_checkpoint = "https://download.openmmlab.com/mmaction/skeleton/posec3d/posec3d_k400.pth"
action_score_thr = 0.4

# use skeleton-based method
use_skeleton_stdet = True
use_skeleton_recog = True

label_map_stdet = "mmaction2/tools/data/ava/label_map.txt"
label_map = "mmaction2/tools/data/kinetics/label_map_k400.txt"


# rgb-based action recognition config
rgb_config = "configs/recognition/tsn/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb.py"
rgb_checkpoint = "https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth"

predict_stepsize = 10  # give out a spatio-temporal detection prediction per n frames
output_stepsize = 1  # show one frame per n frames in the demo, we should have: predict_stepsize % output_stepsize == 0
output_fps = 15  # the fps of demo video output






#args = parse_args()
tmp_dir = tempfile.TemporaryDirectory()
frame_paths, original_frames = frame_extract(
    video, out_dir=tmp_dir.name)
num_frame = len(frame_paths)
h, w, _ = original_frames[0].shape







# Get Human detection results and pose results
human_detections, _ = detection_inference(
    det_config,
    det_checkpoint,
    frame_paths,
    det_score_thr,
    device=device)
torch.cuda.empty_cache()
pose_datasample = None
if use_skeleton_recog or use_skeleton_stdet:
    pose_results, pose_datasample = pose_inference(
        pose_config,
        pose_checkpoint,
        frame_paths,
        human_detections,
        device=device)
torch.cuda.empty_cache()

In [ ]:
print(human_detections)

In [ ]:
print(pose_datasample)

In [ ]:
# resize frames to shortside 256
new_w, new_h = mmcv.rescale_size((w, h), (256, np.Inf))
frames = [mmcv.imresize(img, (new_w, new_h)) for img in original_frames]
w_ratio, h_ratio = new_w / w, new_h / h



# Load spatio-temporal detection label_map
stdet_label_map = load_label_map(label_map_stdet)
rgb_stdet_config = mmengine.Config.fromfile(rgb_stdet_config)
#rgb_stdet_config.merge_from_dict(args.cfg_options)
try:
    if rgb_stdet_config['data']['train']['custom_classes'] is not None:
        stdet_label_map = {
            id + 1: stdet_label_map[cls]
            for id, cls in enumerate(rgb_stdet_config['data']['train']
                                     ['custom_classes'])
        }
except KeyError:
    pass






action_result = None
if use_skeleton_recog:
    print('Use skeleton-based recognition')
    action_result = skeleton_based_action_recognition(
        skeleton_config, skeleton_checkpoint, device, label_map, pose_results, h, w)
else:
    print('Use rgb-based recognition')
    action_result = rgb_based_action_recognition(rgb_config, rgb_checkpoint, device, video, label_map)





stdet_preds = None
if use_skeleton_stdet:
    print('Use skeleton-based SpatioTemporal Action Detection')
    clip_len, frame_interval = 30, 1
    timestamps, stdet_preds = skeleton_based_stdet(predict_stepsize,
                                                   skeleton_config,
                                                   skeleton_stdet_checkpoint,
                                                   device,
                                                   action_score_thr,
                                                   stdet_label_map,
                                                   human_detections,
                                                   pose_results, num_frame,
                                                   clip_len,
                                                   frame_interval, h, w)
    for i in range(len(human_detections)):
        det = human_detections[i]
        det[:, 0:4:2] *= w_ratio
        det[:, 1:4:2] *= h_ratio
        human_detections[i] = torch.from_numpy(det[:, :4]).to(device)

else:
    print('Use rgb-based SpatioTemporal Action Detection')
    for i in range(len(human_detections)):
        det = human_detections[i]
        det[:, 0:4:2] *= w_ratio
        det[:, 1:4:2] *= h_ratio
        human_detections[i] = torch.from_numpy(det[:, :4]).to(device)
    timestamps, stdet_preds = rgb_based_stdet(rgb_stdet_config,
                                              rgb_stdet_checkpoint,
                                              device,
                                              action_score_thr,
                                              predict_stepsize,
                                              frames,
                                              stdet_label_map,
                                              human_detections, w, h,
                                              new_w, new_h, w_ratio,
                                              h_ratio)






stdet_results = []
for timestamp, prediction in zip(timestamps, stdet_preds):
    human_detection = human_detections[timestamp - 1]
    stdet_results.append(
        pack_result(human_detection, prediction, new_h, new_w))

def dense_timestamps(timestamps, n):
    """Make it nx frames."""
    old_frame_interval = (timestamps[1] - timestamps[0])
    start = timestamps[0] - old_frame_interval / n * (n - 1) / 2
    new_frame_inds = np.arange(
        len(timestamps) * n) * old_frame_interval / n + start
    return new_frame_inds.astype(np.int64)

dense_n = int(predict_stepsize / output_stepsize)
output_timestamps = dense_timestamps(timestamps, dense_n)
frames = [
    cv2.imread(frame_paths[timestamp - 1])
    for timestamp in output_timestamps
]

if use_skeleton_recog or use_skeleton_stdet:
    pose_datasample = [
        pose_datasample[timestamp - 1] for timestamp in output_timestamps
    ]

In [ ]:
print(stdet_results)

In [ ]:
vis_frames = visualize(pose_config, frames, stdet_results, pose_datasample,
                       action_result)
vid = mpy.ImageSequenceClip(vis_frames, fps=output_fps)
vid.write_videofile(out_filename)
#tmp_dir.cleanup()

In [ ]:
video = 'data/video_4.mp4'
out_filename = 'data/video_4_out.MP4'

# rgb-based spatio temporal detection config
rgb_stdet_config =  "mmaction2/configs/detection/slowonly/slowonly_kinetics400-pretrained-r101_8xb16-8x8x1-20e_ava21-rgb.py"
rgb_stdet_checkpoint = "https://download.openmmlab.com/mmaction/detection/ava/slowonly_omnisource_pretrained_r101_8x8x1_20e_ava_rgb/slowonly_omnisource_pretrained_r101_8x8x1_20e_ava_rgb_20201217-16378594.pth"

# skeleton-based spatio temporal detection config
skeleton_stdet_checkpoint = "https://download.openmmlab.com/mmaction/skeleton/posec3d/posec3d_ava.pth"

# human detection config
det_config = 'mmaction2/demo/demo_configs/faster-rcnn_r50_fpn_2x_coco_infer.py'
det_checkpoint = 'http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth'
det_score_thr = 0.9
#det_cat_id = 0

# pose estimation config
pose_config = 'mmaction2/demo/demo_configs/td-hm_hrnet-w32_8xb64-210e_coco-256x192_infer.py'
pose_checkpoint = 'https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth'

# action classification config
skeleton_config = "mmaction2/configs/skeleton/posec3d/slowonly_r50_8xb16-u48-240e_ntu60-xsub-keypoint.py"
skeleton_checkpoint = "https://download.openmmlab.com/mmaction/skeleton/posec3d/posec3d_k400.pth"
action_score_thr = 0.4

# use skeleton-based method
use_skeleton_stdet = True
use_skeleton_recog = True

label_map_stdet = "mmaction2/tools/data/ava/label_map.txt"
label_map = "mmaction2/tools/data/kinetics/label_map_k400.txt"


# rgb-based action recognition config
rgb_config = "configs/recognition/tsn/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb.py"
rgb_checkpoint = "https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth"

predict_stepsize = 10  # give out a spatio-temporal detection prediction per n frames
output_stepsize = 1  # show one frame per n frames in the demo, we should have: predict_stepsize % output_stepsize == 0
output_fps = 15  # the fps of demo video output






#args = parse_args()
tmp_dir = tempfile.TemporaryDirectory()
frame_paths, original_frames = frame_extract(
    video, out_dir=tmp_dir.name)
num_frame = len(frame_paths)
h, w, _ = original_frames[0].shape







# Get Human detection results and pose results
human_detections, _ = detection_inference(
    det_config,
    det_checkpoint,
    frame_paths,
    det_score_thr,
    device=device)
torch.cuda.empty_cache()
pose_datasample = None
if use_skeleton_recog or use_skeleton_stdet:
    pose_results, pose_datasample = pose_inference(
        pose_config,
        pose_checkpoint,
        frame_paths,
        human_detections,
        device=device)
torch.cuda.empty_cache()

In [ ]:
print(human_detections)

In [ ]:
print(pose_datasample)

In [ ]:
# resize frames to shortside 256
new_w, new_h = mmcv.rescale_size((w, h), (256, np.Inf))
frames = [mmcv.imresize(img, (new_w, new_h)) for img in original_frames]
w_ratio, h_ratio = new_w / w, new_h / h



# Load spatio-temporal detection label_map
stdet_label_map = load_label_map(label_map_stdet)
rgb_stdet_config = mmengine.Config.fromfile(rgb_stdet_config)
#rgb_stdet_config.merge_from_dict(args.cfg_options)
try:
    if rgb_stdet_config['data']['train']['custom_classes'] is not None:
        stdet_label_map = {
            id + 1: stdet_label_map[cls]
            for id, cls in enumerate(rgb_stdet_config['data']['train']
                                     ['custom_classes'])
        }
except KeyError:
    pass






action_result = None
if use_skeleton_recog:
    print('Use skeleton-based recognition')
    action_result = skeleton_based_action_recognition(
        skeleton_config, skeleton_checkpoint, device, label_map, pose_results, h, w)
else:
    print('Use rgb-based recognition')
    action_result = rgb_based_action_recognition(rgb_config, rgb_checkpoint, device, video, label_map)





stdet_preds = None
if use_skeleton_stdet:
    print('Use skeleton-based SpatioTemporal Action Detection')
    clip_len, frame_interval = 30, 1
    timestamps, stdet_preds = skeleton_based_stdet(predict_stepsize,
                                                   skeleton_config,
                                                   skeleton_stdet_checkpoint,
                                                   device,
                                                   action_score_thr,
                                                   stdet_label_map,
                                                   human_detections,
                                                   pose_results, num_frame,
                                                   clip_len,
                                                   frame_interval, h, w)
    for i in range(len(human_detections)):
        det = human_detections[i]
        det[:, 0:4:2] *= w_ratio
        det[:, 1:4:2] *= h_ratio
        human_detections[i] = torch.from_numpy(det[:, :4]).to(device)

else:
    print('Use rgb-based SpatioTemporal Action Detection')
    for i in range(len(human_detections)):
        det = human_detections[i]
        det[:, 0:4:2] *= w_ratio
        det[:, 1:4:2] *= h_ratio
        human_detections[i] = torch.from_numpy(det[:, :4]).to(device)
    timestamps, stdet_preds = rgb_based_stdet(rgb_stdet_config,
                                              rgb_stdet_checkpoint,
                                              device,
                                              action_score_thr,
                                              predict_stepsize,
                                              frames,
                                              stdet_label_map,
                                              human_detections, w, h,
                                              new_w, new_h, w_ratio,
                                              h_ratio)






stdet_results = []
for timestamp, prediction in zip(timestamps, stdet_preds):
    human_detection = human_detections[timestamp - 1]
    stdet_results.append(
        pack_result(human_detection, prediction, new_h, new_w))

def dense_timestamps(timestamps, n):
    """Make it nx frames."""
    old_frame_interval = (timestamps[1] - timestamps[0])
    start = timestamps[0] - old_frame_interval / n * (n - 1) / 2
    new_frame_inds = np.arange(
        len(timestamps) * n) * old_frame_interval / n + start
    return new_frame_inds.astype(np.int64)

dense_n = int(predict_stepsize / output_stepsize)
output_timestamps = dense_timestamps(timestamps, dense_n)
frames = [
    cv2.imread(frame_paths[timestamp - 1])
    for timestamp in output_timestamps
]

if use_skeleton_recog or use_skeleton_stdet:
    pose_datasample = [
        pose_datasample[timestamp - 1] for timestamp in output_timestamps
    ]

In [ ]:
print(stdet_results)

In [ ]:
vis_frames = visualize(pose_config, frames, stdet_results, pose_datasample,
                       action_result)
vid = mpy.ImageSequenceClip(vis_frames, fps=output_fps)
vid.write_videofile(out_filename)
#tmp_dir.cleanup()

In [ ]:
video = 'data/video_1.mp4'
out_filename = 'data/video_1_out.MP4'

# rgb-based spatio temporal detection config
rgb_stdet_config =  "mmaction2/configs/detection/slowonly/slowonly_kinetics400-pretrained-r101_8xb16-8x8x1-20e_ava21-rgb.py"
rgb_stdet_checkpoint = "https://download.openmmlab.com/mmaction/detection/ava/slowonly_omnisource_pretrained_r101_8x8x1_20e_ava_rgb/slowonly_omnisource_pretrained_r101_8x8x1_20e_ava_rgb_20201217-16378594.pth"

# skeleton-based spatio temporal detection config
skeleton_stdet_checkpoint = "https://download.openmmlab.com/mmaction/skeleton/posec3d/posec3d_ava.pth"

# human detection config
det_config = 'mmaction2/demo/demo_configs/faster-rcnn_r50_fpn_2x_coco_infer.py'
det_checkpoint = 'http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth'
det_score_thr = 0.9
#det_cat_id = 0

# pose estimation config
pose_config = 'mmaction2/demo/demo_configs/td-hm_hrnet-w32_8xb64-210e_coco-256x192_infer.py'
pose_checkpoint = 'https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth'

# action classification config
skeleton_config = "mmaction2/configs/skeleton/posec3d/slowonly_r50_8xb16-u48-240e_ntu60-xsub-keypoint.py"
skeleton_checkpoint = "https://download.openmmlab.com/mmaction/skeleton/posec3d/posec3d_k400.pth"
action_score_thr = 0.4

# use skeleton-based method
use_skeleton_stdet = True
use_skeleton_recog = True

label_map_stdet = "mmaction2/tools/data/ava/label_map.txt"
label_map = "mmaction2/tools/data/kinetics/label_map_k400.txt"


# rgb-based action recognition config
rgb_config = "configs/recognition/tsn/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb.py"
rgb_checkpoint = "https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth"

predict_stepsize = 10  # give out a spatio-temporal detection prediction per n frames
output_stepsize = 1  # show one frame per n frames in the demo, we should have: predict_stepsize % output_stepsize == 0
output_fps = 15  # the fps of demo video output






#args = parse_args()
tmp_dir = tempfile.TemporaryDirectory()
frame_paths, original_frames = frame_extract(
    video, out_dir=tmp_dir.name)
num_frame = len(frame_paths)
h, w, _ = original_frames[0].shape







# Get Human detection results and pose results
human_detections, _ = detection_inference(
    det_config,
    det_checkpoint,
    frame_paths,
    det_score_thr,
    device=device)
torch.cuda.empty_cache()
pose_datasample = None
if use_skeleton_recog or use_skeleton_stdet:
    pose_results, pose_datasample = pose_inference(
        pose_config,
        pose_checkpoint,
        frame_paths,
        human_detections,
        device=device)
torch.cuda.empty_cache()

In [ ]:
print(human_detections)

In [ ]:
print(pose_datasample)

In [ ]:
# resize frames to shortside 256
new_w, new_h = mmcv.rescale_size((w, h), (256, np.Inf))
frames = [mmcv.imresize(img, (new_w, new_h)) for img in original_frames]
w_ratio, h_ratio = new_w / w, new_h / h



# Load spatio-temporal detection label_map
stdet_label_map = load_label_map(label_map_stdet)
rgb_stdet_config = mmengine.Config.fromfile(rgb_stdet_config)
#rgb_stdet_config.merge_from_dict(args.cfg_options)
try:
    if rgb_stdet_config['data']['train']['custom_classes'] is not None:
        stdet_label_map = {
            id + 1: stdet_label_map[cls]
            for id, cls in enumerate(rgb_stdet_config['data']['train']
                                     ['custom_classes'])
        }
except KeyError:
    pass






action_result = None
if use_skeleton_recog:
    print('Use skeleton-based recognition')
    action_result = skeleton_based_action_recognition(
        skeleton_config, skeleton_checkpoint, device, label_map, pose_results, h, w)
else:
    print('Use rgb-based recognition')
    action_result = rgb_based_action_recognition(rgb_config, rgb_checkpoint, device, video, label_map)





stdet_preds = None
if use_skeleton_stdet:
    print('Use skeleton-based SpatioTemporal Action Detection')
    clip_len, frame_interval = 30, 1
    timestamps, stdet_preds = skeleton_based_stdet(predict_stepsize,
                                                   skeleton_config,
                                                   skeleton_stdet_checkpoint,
                                                   device,
                                                   action_score_thr,
                                                   stdet_label_map,
                                                   human_detections,
                                                   pose_results, num_frame,
                                                   clip_len,
                                                   frame_interval, h, w)
    for i in range(len(human_detections)):
        det = human_detections[i]
        det[:, 0:4:2] *= w_ratio
        det[:, 1:4:2] *= h_ratio
        human_detections[i] = torch.from_numpy(det[:, :4]).to(device)

else:
    print('Use rgb-based SpatioTemporal Action Detection')
    for i in range(len(human_detections)):
        det = human_detections[i]
        det[:, 0:4:2] *= w_ratio
        det[:, 1:4:2] *= h_ratio
        human_detections[i] = torch.from_numpy(det[:, :4]).to(device)
    timestamps, stdet_preds = rgb_based_stdet(rgb_stdet_config,
                                              rgb_stdet_checkpoint,
                                              device,
                                              action_score_thr,
                                              predict_stepsize,
                                              frames,
                                              stdet_label_map,
                                              human_detections, w, h,
                                              new_w, new_h, w_ratio,
                                              h_ratio)






stdet_results = []
for timestamp, prediction in zip(timestamps, stdet_preds):
    human_detection = human_detections[timestamp - 1]
    stdet_results.append(
        pack_result(human_detection, prediction, new_h, new_w))

def dense_timestamps(timestamps, n):
    """Make it nx frames."""
    old_frame_interval = (timestamps[1] - timestamps[0])
    start = timestamps[0] - old_frame_interval / n * (n - 1) / 2
    new_frame_inds = np.arange(
        len(timestamps) * n) * old_frame_interval / n + start
    return new_frame_inds.astype(np.int64)

dense_n = int(predict_stepsize / output_stepsize)
output_timestamps = dense_timestamps(timestamps, dense_n)
frames = [
    cv2.imread(frame_paths[timestamp - 1])
    for timestamp in output_timestamps
]

if use_skeleton_recog or use_skeleton_stdet:
    pose_datasample = [
        pose_datasample[timestamp - 1] for timestamp in output_timestamps
    ]

In [ ]:
print(stdet_results)

In [ ]:
vis_frames = visualize(pose_config, frames, stdet_results, pose_datasample,
                       action_result)
vid = mpy.ImageSequenceClip(vis_frames, fps=output_fps)
vid.write_videofile(out_filename)
#tmp_dir.cleanup()

In [ ]:
video = 'data/video_1.mp4'
out_filename = 'data/video_1_out.MP4'

# rgb-based spatio temporal detection config
rgb_stdet_config =  "mmaction2/configs/detection/slowonly/slowonly_kinetics400-pretrained-r101_8xb16-8x8x1-20e_ava21-rgb.py"
rgb_stdet_checkpoint = "https://download.openmmlab.com/mmaction/detection/ava/slowonly_omnisource_pretrained_r101_8x8x1_20e_ava_rgb/slowonly_omnisource_pretrained_r101_8x8x1_20e_ava_rgb_20201217-16378594.pth"

# skeleton-based spatio temporal detection config
skeleton_stdet_checkpoint = "https://download.openmmlab.com/mmaction/skeleton/posec3d/posec3d_ava.pth"

# human detection config
det_config = 'mmaction2/demo/demo_configs/faster-rcnn_r50_fpn_2x_coco_infer.py'
det_checkpoint = 'http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth'
det_score_thr = 0.9
#det_cat_id = 0

# pose estimation config
pose_config = 'mmaction2/demo/demo_configs/td-hm_hrnet-w32_8xb64-210e_coco-256x192_infer.py'
pose_checkpoint = 'https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth'

# action classification config
skeleton_config = "mmaction2/configs/skeleton/posec3d/slowonly_r50_8xb16-u48-240e_ntu60-xsub-keypoint.py"
skeleton_checkpoint = "https://download.openmmlab.com/mmaction/skeleton/posec3d/posec3d_k400.pth"
action_score_thr = 0.4

# use skeleton-based method
use_skeleton_stdet = True
use_skeleton_recog = True

label_map_stdet = "mmaction2/tools/data/ava/label_map.txt"
label_map = "mmaction2/tools/data/kinetics/label_map_k400.txt"


# rgb-based action recognition config
rgb_config = "configs/recognition/tsn/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb.py"
rgb_checkpoint = "https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth"

predict_stepsize = 10  # give out a spatio-temporal detection prediction per n frames
output_stepsize = 1  # show one frame per n frames in the demo, we should have: predict_stepsize % output_stepsize == 0
output_fps = 15  # the fps of demo video output






#args = parse_args()
tmp_dir = tempfile.TemporaryDirectory()
frame_paths, original_frames = frame_extract(
    video, out_dir=tmp_dir.name)
num_frame = len(frame_paths)
h, w, _ = original_frames[0].shape







# Get Human detection results and pose results
human_detections, _ = detection_inference(
    det_config,
    det_checkpoint,
    frame_paths,
    det_score_thr,
    device=device)
torch.cuda.empty_cache()
pose_datasample = None
if use_skeleton_recog or use_skeleton_stdet:
    pose_results, pose_datasample = pose_inference(
        pose_config,
        pose_checkpoint,
        frame_paths,
        human_detections,
        device=device)
torch.cuda.empty_cache()

In [ ]:
# resize frames to shortside 256
new_w, new_h = mmcv.rescale_size((w, h), (256, np.Inf))
frames = [mmcv.imresize(img, (new_w, new_h)) for img in original_frames]
w_ratio, h_ratio = new_w / w, new_h / h



# Load spatio-temporal detection label_map
stdet_label_map = load_label_map(label_map_stdet)
rgb_stdet_config = mmengine.Config.fromfile(rgb_stdet_config)
#rgb_stdet_config.merge_from_dict(args.cfg_options)
try:
    if rgb_stdet_config['data']['train']['custom_classes'] is not None:
        stdet_label_map = {
            id + 1: stdet_label_map[cls]
            for id, cls in enumerate(rgb_stdet_config['data']['train']
                                     ['custom_classes'])
        }
except KeyError:
    pass






action_result = None
if use_skeleton_recog:
    print('Use skeleton-based recognition')
    action_result = skeleton_based_action_recognition(
        skeleton_config, skeleton_checkpoint, device, label_map, pose_results, h, w)
else:
    print('Use rgb-based recognition')
    action_result = rgb_based_action_recognition(rgb_config, rgb_checkpoint, device, video, label_map)





stdet_preds = None
if use_skeleton_stdet:
    print('Use skeleton-based SpatioTemporal Action Detection')
    clip_len, frame_interval = 30, 1
    timestamps, stdet_preds = skeleton_based_stdet(predict_stepsize,
                                                   skeleton_config,
                                                   skeleton_stdet_checkpoint,
                                                   device,
                                                   action_score_thr,
                                                   stdet_label_map,
                                                   human_detections,
                                                   pose_results, num_frame,
                                                   clip_len,
                                                   frame_interval, h, w)
    for i in range(len(human_detections)):
        det = human_detections[i]
        det[:, 0:4:2] *= w_ratio
        det[:, 1:4:2] *= h_ratio
        human_detections[i] = torch.from_numpy(det[:, :4]).to(device)

else:
    print('Use rgb-based SpatioTemporal Action Detection')
    for i in range(len(human_detections)):
        det = human_detections[i]
        det[:, 0:4:2] *= w_ratio
        det[:, 1:4:2] *= h_ratio
        human_detections[i] = torch.from_numpy(det[:, :4]).to(device)
    timestamps, stdet_preds = rgb_based_stdet(rgb_stdet_config,
                                              rgb_stdet_checkpoint,
                                              device,
                                              action_score_thr,
                                              predict_stepsize,
                                              frames,
                                              stdet_label_map,
                                              human_detections, w, h,
                                              new_w, new_h, w_ratio,
                                              h_ratio)






stdet_results = []
for timestamp, prediction in zip(timestamps, stdet_preds):
    human_detection = human_detections[timestamp - 1]
    stdet_results.append(
        pack_result(human_detection, prediction, new_h, new_w))

def dense_timestamps(timestamps, n):
    """Make it nx frames."""
    old_frame_interval = (timestamps[1] - timestamps[0])
    start = timestamps[0] - old_frame_interval / n * (n - 1) / 2
    new_frame_inds = np.arange(
        len(timestamps) * n) * old_frame_interval / n + start
    return new_frame_inds.astype(np.int64)

dense_n = int(predict_stepsize / output_stepsize)
output_timestamps = dense_timestamps(timestamps, dense_n)
frames = [
    cv2.imread(frame_paths[timestamp - 1])
    for timestamp in output_timestamps
]

if use_skeleton_recog or use_skeleton_stdet:
    pose_datasample = [
        pose_datasample[timestamp - 1] for timestamp in output_timestamps
    ]

In [ ]:
pose_datasample

In [ ]:
pose_res=[pose_results[x]['keypoints'] for x in range(len(pose_results))]

In [ ]:
pose_res